In [ ]:
%%capture
!pip install datasets==1.4.1
# !pip install transformers==4.4.0
!pip install transformers['ja']
!pip install torchaudio
!pip install librosa
!pip install jiwer
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install git+https://github.com/miurahr/pykakasi 

In [ ]:
from datasets import load_dataset, load_metric

# common_voice_train = load_dataset("common_voice", "ja", split="train+validation")
common_voice_train = load_dataset("common_voice", "ja", split="train")
common_voice_test = load_dataset("common_voice", "ja", split="test")
print(common_voice_train)

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0)
Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0)


Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 722
})


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
print(common_voice_train)

Dataset({
    features: ['path', 'sentence'],
    num_rows: 722
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "データセット数以上の要素を選ぶことはできません!!!"
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
# 適当に２０個データ表示
show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=20)

,sentence
0,わたしには、約束時間ぎりぎりに、家を出るという、悪い癖がある。
1,きょうの夕飯はカレーです。
2,今も、改めて、このしんと沈んだ抒情に感服する。
3,辞書を使って、英語を勉強します。
4,日本と外国はことばが違います。
5,どの靴を買いますか。
6,舗装という文明の象徴は、しばしば、逆流現象を起こして、人間に害をもたらす。
7,そのあと、数名で、六本木のスシ屋に行った。
8,食堂はあっちの建物にあります。
9,きょうは誰も来ませんでした。


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�‘、。「」〇？…]'

def remove_spesial_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_spesial_characters)
common_voice_test = common_voice_test.map(remove_spesial_characters)
print(common_voice_train)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-d82f4617cfc9aa32.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-4a45edd35a5efbed.arrow


Dataset({
    features: ['path', 'sentence'],
    num_rows: 722
})


In [ ]:
# いらない文字消したやつ（とりあえず句読点消した）
show_random_elements(common_voice_train.remove_columns(["path"]))


,sentence
0,駅から会社までタクシーに乗ります
1,この教室にかばんが九つあります
2,わたしが晩ごはんを食べているとき友だちがうちへ来ました
3,私の指には宝石の指輪はもうはめられません
4,道が分からなかったので地図を見ました
5,イさんはテニスをやりたがっています
6,この花は赤くてきれいです
7,きょうは誰も来ませんでした
8,あの山にはたくさん動物がいます
9,レジェンドシリーズをベースにヨーフリー機構を備えたビューカメラ


In [ ]:
from pykakasi import kakasi 
def jp_conv(batch):
    kakasi_ = kakasi()
    # # カタカナ→ひらがな
    # kakasi_.setMode('K','a') 
    # # 漢字→ひらがな
    # kakasi_.setMode('J','a')
    # kakasi_.setMode('H','a') 

    kakasi_.setMode('K','H') 
    kakasi_.setMode('J','H')
    # 分かち書き
    kakasi_.setMode("s", False)

    conv = kakasi_.getConverter()
    batch["sentence"] = conv.do(batch["sentence"])
    return batch




In [ ]:
common_voice_train = common_voice_train.map(jp_conv)
common_voice_test = common_voice_test.map(jp_conv)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-7a4fbfdd515d2e45.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-c65bed48f0eb25b1.arrow


In [ ]:
# データを全部ひらがなに変えたやつ
show_random_elements(common_voice_train.remove_columns(["path"]))

,sentence
0,れいぞうこのなかにやさいやたまごがあります
1,せんせいからききましたがらいげつからあたらしいせいとがくるそうです
2,いりゅうひんといってはいといっぽんもおちていなかった
3,にっぽんへもどってからそれぞれしゅっせをしているようすであった
4,ゆーざーにもせきにんがあるとのろんりはぼうろんといわざるをえません
5,このりょうりはまずそうです
6,このれすとらんはおおきくてきれいです
7,ほいくしょへまつっこをむかえにいったかえりびょういんへもよったのですっかりおそくなってしまった
8,わたしはそれをりょかんにもってかえった
9,いえへかえったらそうじしようとおもいます


In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_list = sorted(vocab_list)
print(vocab_list)

[' ', 'ぁ', 'あ', 'ぃ', 'い', 'う', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'ゔ', 'ー', '．', 'ｄ', 'ｇ', 'ｐ']


In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

{' ': 0, 'ぁ': 1, 'あ': 2, 'ぃ': 3, 'い': 4, 'う': 5, 'ぇ': 6, 'え': 7, 'ぉ': 8, 'お': 9, 'か': 10, 'が': 11, 'き': 12, 'ぎ': 13, 'く': 14, 'ぐ': 15, 'け': 16, 'げ': 17, 'こ': 18, 'ご': 19, 'さ': 20, 'ざ': 21, 'し': 22, 'じ': 23, 'す': 24, 'ず': 25, 'せ': 26, 'ぜ': 27, 'そ': 28, 'ぞ': 29, 'た': 30, 'だ': 31, 'ち': 32, 'ぢ': 33, 'っ': 34, 'つ': 35, 'づ': 36, 'て': 37, 'で': 38, 'と': 39, 'ど': 40, 'な': 41, 'に': 42, 'ぬ': 43, 'ね': 44, 'の': 45, 'は': 46, 'ば': 47, 'ぱ': 48, 'ひ': 49, 'び': 50, 'ぴ': 51, 'ふ': 52, 'ぶ': 53, 'ぷ': 54, 'へ': 55, 'べ': 56, 'ぺ': 57, 'ほ': 58, 'ぼ': 59, 'ぽ': 60, 'ま': 61, 'み': 62, 'む': 63, 'め': 64, 'も': 65, 'ゃ': 66, 'や': 67, 'ゅ': 68, 'ゆ': 69, 'ょ': 70, 'よ': 71, 'ら': 72, 'り': 73, 'る': 74, 'れ': 75, 'ろ': 76, 'わ': 77, 'を': 78, 'ん': 79, 'ゔ': 80, 'ー': 81, '．': 82, 'ｄ': 83, 'ｇ': 84, 'ｐ': 85}


In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

88

In [ ]:
import json
with open('vocab.json', 'w')as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
common_voice_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/07b0a73f2103df267f566548f7597fe8d75f8d4bdd37b7f556478ae85378bd6a/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_19817895.mp3',
 'sentence': 'よそうがいのじたいにでんりょくがいしゃもちょっぴりこんわくきみだ '}

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-ce27db2f0a66a03e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-73a5463b1154ce8a.arrow


In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-68e8ee2a5726cea1.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-d3b7eac9110d0399.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-a920907ca83da86a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-d2fb36e44cd0c16e.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-f1275f2bb1e7a53d.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-e2349c914d2dc45d.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-fd930da5e68e1f09.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-553d2f064a3b9ccf.arrow


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=np.asanyarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
read_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: げつようびとすいようびときんようびにさっかーのれんしゅうがあります 
Input array shape: (115200,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # サンプリングレートがすべて同じだとsetによりlenが１になる
    assert (
        len(set(batch["sampling_rate"])) == 1
    ),  f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch
# timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)
print(common_voice_train[0])

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-d410c99fdabdd5cc.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-ce78119927630bb0.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-53ae4715b5f79817.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-6d0e0e094f235f38.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-b518ec82962788d0.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-ac2fb9e4fde721d2.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-85a6a9a38cccca6c.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0/cache-ee53d2670c91eda5.arrow


{'input_values': [-4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.98199941020418e-05, -4.981999409495604e-05, -4.981999411779214e-05, -4.981999407379649e-05, -4.981999414133697e-05, -4.9819994022849064e-05, -4.981999423116318e-05, -4.9819993902753703e-05, -4.98199943933358e-05, -4.981999370259964e-05, -4.981999460976037e-05, -4.981999351920943e-05, -4.981999464652614e-05, -4.98199937609093e-05, -4.9819993910022866e-05, -4.981999530340324e-05, -4.9819991165463213e-05, -4.981999974460949e-05, -4.981998448140149e-05, -4.9820009260069525e-05, -4.9819971553480074e-05, -4.982002612924351e-05, -4.981995040442355e-05, -4.982005161580743e-05, -4.981992089486698e-05, -4.98200842

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# data_collator = DataCollatorCTCWithPadding(padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
#   output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    # train_dataset=timit_prepared["train"],
    # eval_dataset=timit_prepared["test"],
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
400,11.665300,3.822974,1.221519,94.126600,6.714000
800,2.961500,1.439067,1.151899,94.838300,6.664000
1200,0.419200,1.464127,1.300633,94.386100,6.696000
1600,0.196500,1.498316,1.185127,94.404700,6.695000
2000,0.134300,1.535715,1.025316,94.514900,6.687000


TrainOutput(global_step=2250, training_loss=2.746637765672472, metrics={'train_runtime': 5539.0409, 'train_samples_per_second': 0.406, 'total_flos': 5.345580017623265e+18, 'epoch': 49.99, 'init_mem_cpu_alloc_delta': 388997120, 'init_mem_gpu_alloc_delta': 1262115840, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -1991413760, 'train_mem_gpu_alloc_delta': 3791938048, 'train_mem_cpu_peaked_delta': 2908581888, 'train_mem_gpu_peaked_delta': 1904590336})

In [ ]:
# model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xlsr-turkish-demo").to("cuda")
# processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xlsr-turkish-demo")
# model = Wav2Vec2ForCTC.from_pretrained("/content/wav2vec2-large-xlsr-turkish-demo").to("cuda")
# print(model)
# print(processor)

In [ ]:
input_dict = processor(common_voice_test[1]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
common_voice_test_transcription = load_dataset("common_voice", "ja",data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Using custom data configuration ja-ad9f7b76efa9f3a0
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ja-ad9f7b76efa9f3a0/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0)


In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[1]["sentence"].lower())


Prediction:
さいふうどんなくしたんのでかうばんなへきます

Reference:
財布をなくしたので、交番へ行きます。
